In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [8]:
def convert_dict_from_update_ticks_to_dataframe(tick_data):
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['ratio'] = df_tick_data['last_tick'] - df_tick_data['last_tick'].shift(1)
    df_tick_data = df_tick_data[['option_name', 'strike', 'bid', 'ask', 'last_tick','ratio','stock_price','deal_type_zone','updated_at']]      
    return df_tick_data


In [15]:
updated_ticks = estrategies.update_quotes(just_last_update=False,mode=InformationType.Offline)
ticks_to_process = updated_ticks[0]

options = []
for i in ticks_to_process:
    options.append(i.__dict__)

dataframe_historical_ticks = convert_dict_from_update_ticks_to_dataframe(options)
dataframe_historical_ticks
ratio_statistic_finder = dataframe_historical_ticks[['updated_at','option_name','ratio']]
ratio_statistic_finder = ratio_statistic_finder.set_index('updated_at')
ratio_statistic_finder = ratio_statistic_finder.between_time('10:10', '17:00')
ratio_statistic_finder = ratio_statistic_finder.reset_index()
ratio_statistic_finder = ratio_statistic_finder.pivot(index='updated_at', columns='option_name', values='ratio')
ratio_statistic_finder = ratio_statistic_finder.dropna(axis=1)
ratio_statistic_finder

ratio_statistic_data = {}
for (column, value) in ratio_statistic_finder.iteritems():
    ratio_statistic_data.update({ column: { "Mean": ratio_statistic_finder[column].mean(), "StdDev": ratio_statistic_finder[column].std(), "2xStdDev": (ratio_statistic_finder[column].std()*2)  }}) 

ratio_statistic_dataframe = pd.DataFrame.from_dict(ratio_statistic_data)
ratio_statistic_dataframe = ratio_statistic_dataframe.transpose()

updated_ticks = estrategies.update_quotes(just_last_update=True,mode=InformationType.Real_Time)
ticks_to_process = updated_ticks[0]
options_updated = []
for i in ticks_to_process:
    options_updated.append(i.__dict__)

df_options_updated = convert_dict_from_update_ticks_to_dataframe(options_updated)
df_options_updated.reset_index()
df_options_updated = df_options_updated.set_index('option_name')
df_options_updated = pd.merge(df_options_updated, ratio_statistic_dataframe, left_index=True, right_index=True)

df_options_updated['above_mean'] = abs(df_options_updated['ratio']) > abs(df_options_updated['Mean'])
df_options_updated['above_2x_std_dev'] = abs(df_options_updated['ratio']) > abs(abs(df_options_updated['Mean']) + df_options_updated['2xStdDev'])
df_options_updated['a2xStdDev_Price'] = abs(abs(df_options_updated['Mean']) + df_options_updated['2xStdDev'])
df_options_updated = df_options_updated.drop(columns=['updated_at','StdDev','2xStdDev'])
df_options_updated[df_options_updated['above_mean'] == True]

,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,Mean,above_mean,above_2x_std_dev,a2xStdDev_Price


In [16]:
df_options_updated

,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,Mean,above_mean,above_2x_std_dev,a2xStdDev_Price
PETRH269,26.21,1.67,1.70,1.69,-0.16,26.98,ITM,0.998512,False,False,3.097781
PETRH254,26.46,1.52,1.54,1.52,-0.17,26.98,ITM,-0.180128,False,False,0.240989
PETRH261,27.21,1.12,1.13,1.12,-0.14,26.98,OTM,-0.156488,False,False,0.206202
PETRH247,27.96,0.79,0.80,0.79,-0.10,26.98,OTM,-0.129974,False,False,0.179997
PETRH289,28.21,0.70,0.71,0.71,-0.08,26.98,OTM,-0.110884,False,False,0.140364
PETRH279,28.46,0.61,0.63,0.61,-0.10,26.98,OTM,-0.107738,False,False,0.135849
PETRH294,28.71,0.54,0.55,0.54,-0.07,26.98,OTM,-0.096091,False,False,0.125749
PETRH243,28.96,0.47,0.48,0.46,-0.08,26.98,OTM,-0.087738,False,False,0.114686
PETRH299,29.21,0.41,0.42,0.41,-0.05,26.98,OTM,-0.083150,False,False,0.111110


In [17]:
dataframe_historical_ticks

,option_name,strike,bid,ask,last_tick,ratio,stock_price,deal_type_zone,updated_at
0,PETRH269,26.21,2.46,2.53,2.38,NaN,28.09,ITM,2021-07-14 10:09:44.647
1,PETRH254,26.46,2.28,2.34,2.19,-0.19,28.09,ITM,2021-07-14 10:09:44.647
2,PETRH274,26.71,2.11,2.16,2.02,-0.17,28.09,ITM,2021-07-14 10:09:44.647
3,PETRH276,26.96,1.92,1.98,1.83,-0.19,28.09,ITM,2021-07-14 10:09:44.647
4,PETRH261,27.21,1.76,1.82,1.81,-0.02,28.09,ITM,2021-07-14 10:09:44.647
...,...,...,...,...,...,...,...,...,...
47769,PETRH279,28.46,0.61,0.63,0.61,-0.10,26.98,OTM,2021-07-15 16:20:33.847
47770,PETRH294,28.71,0.54,0.55,0.54,-0.07,26.98,OTM,2021-07-15 16:20:33.847
47771,PETRH243,28.96,0.47,0.49,0.46,-0.08,26.98,OTM,2021-07-15 16:20:33.847
47772,PETRH299,29.21,0.41,0.42,0.41,-0.05,26.98,OTM,2021-07-15 16:20:33.847
